# ✅ Databento + SQLite 示例 Notebook
完整流程，使用 `databento` 从 Nasdaq（XNAS）获取历史 OHLCV 数据，保存到 SQLite，并避免 KeyError 错误。

In [41]:
!pip install databento pandas

In [42]:
import databento as db
import sqlite3
import pandas as pd

# ✅ Your Databento API Key
API_KEY = "db-HJbwXgmEyHqvLp8PLsCdCHmg4KfKp"

# ✅ SQLite database file
DB_FILE = ' Russell 2000.db'

# ✅ Initialize Databento Historical client
client = db.Historical(API_KEY)

def init_db():
    """
    Initialize the SQLite database and create the market_data table if it doesn't exist.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS market_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            symbol TEXT,
            ts_event TEXT,
            open REAL,
            high REAL,
            low REAL,
            close REAL,
            volume REAL
        )
    ''')
    conn.commit()
    cursor.close()
    conn.close()
    print("✅ SQLite table has been created successfully")


In [43]:
def fetch_historical_data(symbol, start, end):
    """
    Fetch historical OHLCV data for a given symbol and time range from Databento.
    """
    try:
        data = client.timeseries.get_range(
            dataset="XNAS.ITCH",    # Nasdaq ITCH dataset
            schema="ohlcv-1m",      # 1-minute OHLCV schema
            symbols=symbol,
            start=start,
            end=end
        )
        df = data.to_df()
        print(df.columns)  # Print columns to verify schema
        print(df.head())   # Print first few rows for quick preview
        return df
    except Exception as e:
        print(f"⚠️ Failed to fetch data for {symbol}: {e}")
        return pd.DataFrame()


In [44]:
def save_data_sqlite(symbol, df):
    """
    Save the historical data DataFrame into the SQLite database.
    Each row will be inserted with symbol, timestamp, and OHLCV values.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    for _, row in df.iterrows():
        cursor.execute(
            '''
            INSERT INTO market_data (symbol, ts_event, open, high, low, close, volume)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            ''',
            (
                symbol,
                str(row.name),  # ✅ Use DataFrame index as the timestamp
                row["open"],
                row["high"],
                row["low"],
                row["close"],
                row["volume"]
            )
        )
    conn.commit()
    cursor.close()
    conn.close()


In [45]:
def main():
    """
    Main execution function.
    Initializes the database, fetches historical data for each symbol, 
    and saves it into the SQLite database.
    """
    init_db()
    symbols = ["IWM"]  # Use IWM as the Russell 2000 Index representative
    for symbol in symbols:
        print(f"🔎 Processing: {symbol}")
        df = fetch_historical_data(symbol, start="2023-07-07", end="2023-07-11")
        if not df.empty:
            save_data_sqlite(symbol, df)
            print(f"✅ {symbol} data saved successfully.")
        else:
            print(f"❌ No data found for {symbol}.")
    print("🎉 All tasks completed.")

main()


✅ SQLite table has been created successfully
🔎 Processing: IWM
Index(['rtype', 'publisher_id', 'instrument_id', 'open', 'high', 'low',
       'close', 'volume', 'symbol'],
      dtype='object')
                           rtype  publisher_id  instrument_id    open    high  \
ts_event                                                                        
2023-07-07 08:00:00+00:00     33             2           5597  182.11  182.11   
2023-07-07 08:03:00+00:00     33             2           5597  181.97  181.97   
2023-07-07 08:04:00+00:00     33             2           5597  182.05  182.05   
2023-07-07 08:09:00+00:00     33             2           5597  182.00  182.00   
2023-07-07 08:18:00+00:00     33             2           5597  182.35  182.35   

                              low   close  volume symbol  
ts_event                                                  
2023-07-07 08:00:00+00:00  182.11  182.11     100    IWM  
2023-07-07 08:03:00+00:00  181.97  181.97      75    IWM  
20

In [46]:
def query_data():
    conn = sqlite3.connect(DB_FILE)
    df = pd.read_sql("SELECT * FROM market_data ORDER BY ts_event DESC LIMIT 10", conn)
    conn.close()
    return df

query_data()

,id,symbol,ts_event,open,high,low,close,volume
0,1137,IWM,2023-07-10 23:58:00+00:00,188.27,188.33,188.27,188.33,1923.0
1,1136,IWM,2023-07-10 23:57:00+00:00,188.28,188.29,188.27,188.27,940.0
2,1135,IWM,2023-07-10 23:49:00+00:00,188.24,188.26,188.24,188.26,10.0
3,1134,IWM,2023-07-10 23:46:00+00:00,188.23,188.23,188.23,188.23,344.0
4,1133,IWM,2023-07-10 23:45:00+00:00,188.20,188.20,188.20,188.20,5.0
5,1132,IWM,2023-07-10 23:42:00+00:00,188.16,188.16,188.16,188.16,80.0
6,1131,IWM,2023-07-10 23:34:00+00:00,188.14,188.14,188.14,188.14,10.0
7,1130,IWM,2023-07-10 23:30:00+00:00,188.24,188.24,188.24,188.24,109.0
8,1129,IWM,2023-07-10 23:29:00+00:00,188.23,188.23,188.23,188.23,263.0
9,1128,IWM,2023-07-10 23:10:00+00:00,188.24,188.24,188.24,188.24,50.0
